# Introduction

The purpose of this .ipynb is to generate motif-pair in-silico perturbations across canonical motifs and their variants. This will allow us to assess whether motif pairs have preferential distances or spacings between them.

# Computational setup

In [1]:
import warnings
warnings.filterwarnings("ignore")
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

#Packages
import os
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm
from pybedtools import BedTool

# Settings
os.chdir(f'/n/projects/mw2098/collaboration/for_srivastava/analysis/CM_vs_MN_merged_bias_exp_peaks/')
pd.set_option('display.max_columns', 100)

# Custom commands
sys.path.insert(0, f'/n/projects/mw2098/shared_code/bpnet/scripts')
from data_format_functions import myround, myfloor, myceiling

# function to return key for any value 
def get_key(val, my_dict): 
    for key, value in my_dict.items(): 
        if val == value: 
            return key 
    return "key doesn't exist"

Using TensorFlow backend.
2021-07-16 15:53:47,190 [WARNING] git-lfs not installed
2021-07-16 15:53:48,749 [INFO] Note: detected 80 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2021-07-16 15:53:48,751 [INFO] Note: NumExpr detected 80 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-07-16 15:53:48,752 [INFO] NumExpr defaulting to 8 threads.


In [2]:
#Pre-existing variables
fasta_file = f'/n/projects/mw2098/genomes/hg19/hg19.fa'
model_prefix = 'seq_width1000-lr0.001-lambda100-n_dil_layers9-conv_kernel_size7-tconv_kernel_size7-filters64'
prefix = 'CM_vs_MN_merged_bias_exp_peaks'
tasks = ['I_WT_D6CM','I_WT_S3MN']

# Determined variables
model_dir = f'../../models/{prefix}/{model_prefix}'
modisco_dir = f'../../modisco/{prefix}/{model_prefix}'


motif_seqs = {'NKX2.5': 'TTAAGTG',
              'NKX2.5-alt': 'CTAAGTG',
              'GATA': 'AGATAAG',
              'ISL1': 'CTAATGG',
              'LHX': 'TAATTCAATTA',
              'NeuroD': 'CAGCTG',
              'EBF1': 'TCCCCAGGGA',
              'Onecut2': 'TCCCCAGGGA'
             }

In [3]:
!mkdir -p ../../figures/{prefix}/8_in_silico_distances/sum
!mkdir -p tsv/insilico_perturb

# Define motif pairs

Here, we will look at each homotypic and heterotypic motif pair.

In [4]:
from itertools import permutations 
motif_perms = list(permutations(motif_seqs.keys(), 2))
motif_perms.extend([(i,i) for i in motif_seqs.keys()])

# Import BPNet model

Load the BPNet model that was trained.

In [5]:
from bpnet.BPNet import BPNetSeqModel
from bpnet.utils import create_tf_session
create_tf_session('0', .4)
bpn = BPNetSeqModel.from_mdir(model_dir)  # wrap SeqModel to BPNetSeqModel to get `sim_pred` method

TF-MoDISco is using the TensorFlow backend.


# Define functions

We want to define functions that will allow us to generate simulated, injected versions of motif pairs and format them into pd.dfs that can be plotted.

In [6]:
from collections import OrderedDict
from bpnet.plot.tracks import filter_tracks
from bpnet.utils import flatten, unflatten
import numpy as np
from copy import deepcopy
import pandas as pd
from scipy.stats import entropy
import random
from concise.preprocessing import encodeDNA
from bpnet.plot.tracks import plot_tracks
from kipoi_utils.data_utils import get_dataset_item, numpy_collate_concat
from bpnet.functions import mean, softmax
from tqdm import tqdm
from kipoi_utils.utils import unique_list
import matplotlib.pyplot as plt
from bpnet.modisco.core import Seqlet

from bpnet.simulate import simmetric_kl

#Get results of bpnet.BPNet.sim_pred into a tidy pd.df
def sim_pred_to_df(sim_pred_profiles):
    dfp_ref = pd.DataFrame()
    for k in sim_pred_profiles.keys():
        df = pd.DataFrame(sim_pred_profiles[k]).reset_index()
        df['task'] = os.path.basename(k)
        df.columns = ['position','prediction','task']
        dfp_ref = dfp_ref.append(df)
    return dfp_ref

#Get results of bpnet.simulate.generate_sim into a tidy pd.df
def generate_sim_to_df(generate_sim_profiles):
    dfp_alt = pd.DataFrame()
    for d in range(len(generate_sim_profiles)):
        dfp = sim_pred_to_df(generate_sim_profiles[d][1]['profile'])
        dfp['distance'] = generate_sim_profiles[d][0]
        dfp_alt = dfp_alt.append(dfp)
    return dfp_alt

def generate_sim(bpnet, central_motif, side_motif, side_distances,
                 center_coords=[450, 550], repeat=128, contribution=['count', 'profile'], correct=False):
    outl = []
    tasks = bpnet.tasks
    seqlen = bpnet.input_seqlen()
    # ref_preds = sim_pred(model, central_motif)
    ref_preds = unflatten(bpnet.sim_pred(central_motif,
                                         repeat=repeat,
                                         contribution=contribution), "/")
    none_preds = unflatten(bpnet.sim_pred('', '', [],
                                          repeat=repeat, contribution=contribution), "/")

    alt_profiles = []
    for dist in tqdm(side_distances):
        # alt_preds = sim_pred(model, central_motif, side_motif, [dist])
        alt_preds = unflatten(bpnet.sim_pred(central_motif, side_motif, [dist],
                                             repeat=repeat, contribution=contribution), "/")
        if correct:
            # Correct for the 'shoulder' effect
            #
            # this performs: AB - (B - 0)
            # Where:
            # - AB: contains both, central and side_motif
            # - B : contains only side_motif
            # - 0 : doesn't contain any motif
            edge_only_preds = unflatten(bpnet.sim_pred('', side_motif, [dist],
                                                       repeat=repeat, contribution=contribution), "/")

            alt_preds_f = flatten(alt_preds, '/')
            # ref_preds_f = flatten(ref_preds, '/')
            none_preds_f = flatten(none_preds, "/")
            # substract the other counts
            alt_preds = unflatten({k: alt_preds_f[k] - v + none_preds_f[k]
                                   for k, v in flatten(edge_only_preds, "/").items()}, "/")
            # ref_preds = unflatten({k: ref_preds_f[k] - v  for k,v in flatten(none_preds, "/").items()}, "/")
        alt_profiles.append((dist, alt_preds))

        # This normalizes the score by `A` finally yielding:
        # (AB - B + 0) / A
        scores = get_scores(ref_preds, alt_preds, tasks, central_motif, seqlen, center_coords)

        # compute the distance metrics
        for task in bpnet.tasks:
            d = scores[task]

            # book-keeping
            d['task'] = task
            d['central_motif'] = central_motif
            d['side_motif'] = side_motif
            d['position'] = dist
            d['distance'] = dist - seqlen // 2

            outl.append(d)

    return pd.DataFrame(outl), alt_profiles

def get_scores(ref_pred, alt_pred, tasks, motif, seqlen, center_coords):
    d = {}
    cstart, cend = center_coords
    for task in tasks:
        # profile - use the filtered tracks
        d[task] = flatten({"profile": profile_sim_metrics(ref_pred['profile'][task][cstart:cend],
                                                          alt_pred['profile'][task][cstart:cend])}, "/")

        # contribution scores - use the central motif region
        if 'contrib' in ref_pred:
            for contrib_score in ref_pred["contrib"][task]:
                contrib, contrib_frac = contrib_sim_metrics(ref_pred["contrib"][task][contrib_score],
                                                            alt_pred["contrib"][task][contrib_score], motif, seqlen)
                d[task] = {f"contrib/{contrib_score}": contrib, f"contrib/{contrib_score}_frac": contrib_frac, **d[task]}
    return d

def profile_sim_metrics(ref, alt, pc=0):
    d = {}
    d['simmetric_kl'] = simmetric_kl(ref, alt).mean() - simmetric_kl(ref, ref).mean()
    d['counts'] = alt.sum()
    d['counts_frac'] = (alt.sum() + pc) / (ref.sum() + pc)
    d['max'] = alt.max()
    d['max_frac'] = (alt.max() + pc) / (ref.max() + pc)

    max_idx = np.argmax(ref, axis=0)
    d['counts_max_ref'] = alt[max_idx, [0]].sum()
    d['counts_max_ref_frac'] = (d['counts_max_ref'] + pc) / (ref[max_idx, [0]].sum() + pc)
    return d

#Get perturbations and convert into tidy.df (dfs = summary, dfp = profiles)
def get_sim(motif_comb, motif_seqs, side_distances = np.arange(505, 700), center_coords = [475, 525]):
    dfs, profiles = generate_sim(bpn, central_motif=motif_seqs[motif_comb[0]], side_motif=motif_seqs[motif_comb[1]], 
                       side_distances=side_distances, center_coords=center_coords, contribution=[], correct=True, repeat = 128)
    dfs.central_motif = motif_comb[0]
    dfs.side_motif = motif_comb[1]
    
    #Get reference profiles
    profiles_ref = bpn.sim_pred(motif_seqs[motif_comb[0]], repeat = 128)
    dfp_ref = sim_pred_to_df(profiles_ref)
    dfp_ref['distance'] = 'Reference'

    #Get perturbed profiles
    dfp_alt = generate_sim_to_df(profiles)

    #Combine
    dfp = dfp_ref.append(dfp_alt)

    return dfs, dfp

# Generate distance-separated motif pair measurements

After knocking out the 'side' injected motif, measure the effect on the 'central' motif. Do this for varying distances to investigate the effects of motif pair spacing between each set.

In [7]:
#%%script false --no-raise-error
dfs_all = pd.DataFrame()
for perm in motif_perms:
    dfs, _ = get_sim(motif_seqs = motif_seqs, motif_comb=perm, side_distances = np.arange(510, 620))
    dfs_all = dfs_all.append(dfs)
dfs_all.to_csv(f'tsv/insilico_perturb/insilico_summaries.tsv.gz', sep = '\t')

100%|██████████| 110/110 [00:32<00:00,  3.34it/s]


Read in the summaries after they are generated and assign motif pair names and information for simultaneous plotting.

In [8]:
dfs_all = pd.read_csv(f'tsv/insilico_perturb/insilico_summaries.tsv.gz', sep = '\t')
dfs_all.head(n=5)

,Unnamed: 0,profile/simmetric_kl,profile/counts,profile/counts_frac,profile/max,profile/max_frac,profile/counts_max_ref,profile/counts_max_ref_frac,task,central_motif,side_motif,position,distance
0,0,0.000289,0.206230,1.164459,0.004500,1.212017,0.004500,1.212017,I_WT_D6CM,NKX2.5,NKX2.5-alt,510,10
1,1,0.000415,0.077576,0.968362,0.001631,0.974014,0.001591,0.950136,I_WT_S3MN,NKX2.5,NKX2.5-alt,510,10
2,2,0.000478,0.247855,1.399492,0.005421,1.460272,0.005417,1.458996,I_WT_D6CM,NKX2.5,NKX2.5-alt,511,11
3,3,0.000268,0.076581,0.955940,0.001590,0.949813,0.001583,0.945259,I_WT_S3MN,NKX2.5,NKX2.5-alt,511,11
4,4,0.000214,0.245519,1.386302,0.005375,1.447714,0.005375,1.447714,I_WT_D6CM,NKX2.5,NKX2.5-alt,512,12


In [9]:
#Assign motif_pair names for faceting
motif_pair = []
for i in range(dfs_all.shape[0]):
    if dfs_all.central_motif.iloc[i] > dfs_all.side_motif.iloc[i]:
        val = dfs_all.side_motif.iloc[i] + '_' + dfs_all.central_motif.iloc[i]
        motif_pair.append(val)
    else:
        val = dfs_all.central_motif.iloc[i] + '_' + dfs_all.side_motif.iloc[i]
        motif_pair.append(val)
dfs_all['motif_pair'] = motif_pair
dfs_all['motif_pair_raw'] = dfs_all.central_motif + '_' + dfs_all.side_motif

#Make categorigal variable column
dfs_all['task'] = pd.Categorical(dfs_all['task'], categories = ['I_WT_D6CM','I_WT_S3MN'], ordered = False)
dfs_all.head(n=4)

,Unnamed: 0,profile/simmetric_kl,profile/counts,profile/counts_frac,profile/max,profile/max_frac,profile/counts_max_ref,profile/counts_max_ref_frac,task,central_motif,side_motif,position,distance,motif_pair,motif_pair_raw
0,0,0.000289,0.206230,1.164459,0.004500,1.212017,0.004500,1.212017,I_WT_D6CM,NKX2.5,NKX2.5-alt,510,10,NKX2.5_NKX2.5-alt,NKX2.5_NKX2.5-alt
1,1,0.000415,0.077576,0.968362,0.001631,0.974014,0.001591,0.950136,I_WT_S3MN,NKX2.5,NKX2.5-alt,510,10,NKX2.5_NKX2.5-alt,NKX2.5_NKX2.5-alt
2,2,0.000478,0.247855,1.399492,0.005421,1.460272,0.005417,1.458996,I_WT_D6CM,NKX2.5,NKX2.5-alt,511,11,NKX2.5_NKX2.5-alt,NKX2.5_NKX2.5-alt
3,3,0.000268,0.076581,0.955940,0.001590,0.949813,0.001583,0.945259,I_WT_S3MN,NKX2.5,NKX2.5-alt,511,11,NKX2.5_NKX2.5-alt,NKX2.5_NKX2.5-alt


Plot the motif pair summaries at their respective distances. 

In [10]:
dfs_all.motif_pair.unique()

array(['NKX2.5_NKX2.5-alt', 'GATA_NKX2.5', 'ISL1_NKX2.5', 'LHX_NKX2.5',
       'NKX2.5_NeuroD', 'EBF1_NKX2.5', 'NKX2.5_Onecut2',
       'GATA_NKX2.5-alt', 'ISL1_NKX2.5-alt', 'LHX_NKX2.5-alt',
       'NKX2.5-alt_NeuroD', 'EBF1_NKX2.5-alt', 'NKX2.5-alt_Onecut2',
       'GATA_ISL1', 'GATA_LHX', 'GATA_NeuroD', 'EBF1_GATA',
       'GATA_Onecut2', 'ISL1_LHX', 'ISL1_NeuroD', 'EBF1_ISL1',
       'ISL1_Onecut2', 'LHX_NeuroD', 'EBF1_LHX', 'LHX_Onecut2',
       'EBF1_NeuroD', 'NeuroD_Onecut2', 'EBF1_Onecut2', 'NKX2.5_NKX2.5',
       'NKX2.5-alt_NKX2.5-alt', 'GATA_GATA', 'ISL1_ISL1', 'LHX_LHX',
       'NeuroD_NeuroD', 'EBF1_EBF1', 'Onecut2_Onecut2'], dtype=object)

In [11]:
import plotnine
from plotnine import *
plotnine.options.figure_size = (8, 2)

for pair in tqdm(dfs_all.motif_pair.unique()):
    df = dfs_all[dfs_all.motif_pair==pair]
    df['profile/counts_max_ref_log2'] = np.log2(df['profile/counts_max_ref_frac'])
    df['profile/counts_log2'] = np.log2(df['profile/counts_frac'])
    
    gmax = (ggplot(data = df, mapping = aes(x='distance', y='profile/counts_max_ref_log2')) + 
     geom_line(aes(color = 'task'))+ #, linetype = 'featured_task')) + 
     facet_wrap('~motif_pair_raw', nrow = 1) +
     geom_hline(yintercept=0, alpha=0.2) + 
     scale_x_continuous(breaks = range(0,120,20), name = 'Center-to-center distance (bp)')+
     scale_y_continuous(name = 'Max. log2(FC) of preds.')+
     theme_minimal())
    gmax.save(f'../../figures/{prefix}/8_in_silico_distances/max/insilico_perturb_max_{pair}.png', height = 2, width = 8)
    gmax.save(f'../../figures/{prefix}/8_in_silico_distances/max/insilico_perturb_max_{pair}.pdf', height = 2, width = 8)
    
    gsum = (ggplot(data = df, mapping = aes(x='distance', y='profile/counts_log2')) + 
     geom_line(aes(color = 'task'))+ 
     facet_wrap('~motif_pair_raw', nrow = 1) +
     geom_hline(yintercept=0, alpha=0.2) + 
     scale_x_continuous(breaks = range(0,120,20), name = 'Center-to-center distance (bp)')+
     scale_y_continuous(name = 'Sum log2(fc) of preds.')+

     theme_minimal())
    gsum.save(f'../../figures/{prefix}/8_in_silico_distances/sum/insilico_perturb_sum_{pair}.png', height = 2, width = 8)
    gsum.save(f'../../figures/{prefix}/8_in_silico_distances/sum/insilico_perturb_sum_{pair}.pdf', height = 2, width = 8)
    gsum

100%|██████████| 36/36 [01:05<00:00,  1.81s/it]
